<a href="https://colab.research.google.com/github/anuragbisht12/Machine-Learning-Algorithms/blob/master/MultiLabel_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import scipy
import numpy
import pandas as pd
from scipy.io import arff
data, meta = scipy.io.arff.loadarff('/content/Dataset/yeast-train.arff')
df_train = pd.DataFrame(data)
df_train=df_train.astype(numpy.float64)

In [62]:
data1, meta1 = scipy.io.arff.loadarff('/content/Dataset/yeast-test.arff')
df_test = pd.DataFrame(data1)
df_test=df_test.astype(numpy.float64)

In [63]:
y_train=df_train.iloc[:,-14:-1]
X_train=df_train.iloc[:,:103]
y_test=df_test.iloc[:,-14:-1]
X_test=df_test.iloc[:,:103]

In [42]:
df.columns

Index(['Att1', 'Att2', 'Att3', 'Att4', 'Att5', 'Att6', 'Att7', 'Att8', 'Att9',
       'Att10',
       ...
       'Class5', 'Class6', 'Class7', 'Class8', 'Class9', 'Class10', 'Class11',
       'Class12', 'Class13', 'Class14'],
      dtype='object', length=117)

In [56]:
df.head()

,Att1,Att2,Att3,Att4,Att5,Att6,Att7,Att8,Att9,Att10,Att11,Att12,Att13,Att14,Att15,Att16,Att17,Att18,Att19,Att20,Att21,Att22,Att23,Att24,Att25,Att26,Att27,Att28,Att29,Att30,Att31,Att32,Att33,Att34,Att35,Att36,Att37,Att38,Att39,Att40,...,Att78,Att79,Att80,Att81,Att82,Att83,Att84,Att85,Att86,Att87,Att88,Att89,Att90,Att91,Att92,Att93,Att94,Att95,Att96,Att97,Att98,Att99,Att100,Att101,Att102,Att103,Class1,Class2,Class3,Class4,Class5,Class6,Class7,Class8,Class9,Class10,Class11,Class12,Class13,Class14
0,0.093700,0.139771,0.062774,0.007698,0.083873,-0.119156,0.073305,0.005510,0.027523,0.043477,-0.024946,0.061221,0.147377,0.082805,-0.011043,-0.001974,-0.147627,0.123673,0.005631,0.030659,0.005683,0.053414,0.069935,0.033555,0.105394,-0.013519,0.119103,-0.057485,-0.028780,0.144546,0.101353,0.024763,0.011344,0.057414,-0.104664,-0.019050,-0.113221,-0.209969,-0.107153,0.417066,...,-0.081028,0.105889,-0.004947,-0.039124,-0.007124,0.017693,-0.068304,-0.021874,-0.011525,-0.029076,0.026808,-0.043047,0.011630,0.008882,-0.012356,-0.052636,0.039048,-0.018712,-0.034711,-0.038675,-0.039102,0.017429,-0.052659,-0.042402,0.118473,0.125632,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.022711,-0.050504,-0.035691,-0.065434,-0.084316,-0.378560,0.038212,0.085770,0.182613,-0.055544,0.029267,0.042597,-0.107352,0.019207,0.047062,-0.027285,0.175346,-0.103701,0.012758,0.058121,-0.060435,0.073053,0.046101,-0.039845,0.028614,0.130573,-0.011646,-0.138608,0.072822,-0.008652,-0.092463,-0.018793,0.028554,0.014002,0.047983,0.049252,0.191805,0.212651,0.145249,0.076901,...,0.017657,-0.041264,-0.010627,-0.023341,0.018021,-0.032040,-0.070248,-0.008586,-0.013087,0.006237,-0.035943,-0.022597,-0.044843,-0.042019,-0.003376,0.004003,-0.001198,0.030594,-0.021814,0.010430,-0.013809,-0.009248,-0.027318,-0.014191,0.022783,0.123785,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2,-0.090407,0.021198,0.208712,0.102752,0.119315,0.041729,-0.021728,0.019603,-0.063853,-0.053756,0.078468,0.130276,0.082742,-0.041696,-0.028589,0.008653,-0.107325,-0.136729,-0.190598,-0.153106,-0.096053,-0.032872,-0.008912,0.042825,0.075615,-0.006747,-0.087992,-0.089286,-0.078429,-0.206253,-0.063610,-0.067006,-0.079174,0.088420,0.094088,-0.091405,-0.102339,-0.121139,-0.072064,0.028891,...,-0.007941,0.080493,-0.082171,-0.099567,-0.080164,0.315935,-0.125758,-0.096519,-0.097738,0.255025,-0.064278,-0.057198,0.131774,0.200930,-0.067795,-0.103332,0.195777,0.022294,0.012583,0.002233,-0.002072,-0.010981,0.007615,-0.063378,-0.084181,-0.034402,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,-0.085235,0.009540,-0.013228,0.094063,-0.013592,-0.030719,-0.116062,-0.131674,-0.165448,-0.123053,-0.088342,-0.010670,0.030590,0.120052,0.129741,0.105897,0.022556,0.009029,-0.177667,0.124952,-0.089325,-0.035157,-0.071450,0.006351,0.066697,0.015784,-0.007242,-0.027891,-0.062685,0.072125,0.069402,-0.131082,-0.066887,0.034905,0.075313,-0.020978,-0.091263,-0.141892,-0.075490,-0.135369,...,-0.097610,-0.012570,-0.043560,-0.068257,-0.057833,-0.023624,-0.026421,0.002439,0.032469,-0.014280,-0.038554,-0.046454,0.052214,0.043700,0.041239,-0.080285,0.001189,-0.066241,-0.046999,-0.066604,-0.055773,-0.041941,0.051066,0.004976,0.193972,0.131866,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
4,-0.088765,-0.026743,0.002075,-0.043819,-0.005465,0.004306,-0.055865,-0.071484,-0.159025,-0.111348,-0.113015,-0.151546,-0.224416,-0.193918,-0.097713,-0.078790,-0.045703,0.061530,0.168146,-0.059478,-0.092706,-0.027548,-0.106203,-0.165190,-0.056432,-0.074915,-0.108129,-0.093601,-0.031025,-0.032656,-0.017192,-0.077063,0.074747,0.021375,0.046370,0.144823,0.136922,0.093619,0.127604,-0.027046,...,-0.101195,0.025472,-0.067447,-0.060830,-0.037247,0.120840,-0.009208,-0.079342,0.014329,0.044347,-0.018380,-0.029290,0.435292,0.227508,-0.064453,-0.086068,-0.035045,-0.080882,0.028468,-0.073576,0.050630,0.084832,-0.019570,-0.021650,-0.068326,-0.091155,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Here, Att represents the attributes or the independent variables and Class represents the target variables.
from sklearn.datasets import make_multilabel_classification

# this will generate a random multi-label dataset
X, y = make_multilabel_classification(sparse = True, n_labels = 20,
return_indicator = 'sparse', allow_unlabeled = False)

In [15]:
# sparse: If True, returns a sparse matrix, where sparse matrix means a matrix having a large number of zero elements.

# n_labels:  The average number of labels for each instance.

# return_indicator: If ‘sparse’ return Y in the sparse binary indicator format.

# allow_unlabeled: If True, some instances might not belong to any class.

In [16]:
# three methods to solve a multi-label classification problem, namely:

# Problem Transformation
# Adapted Algorithm
# Ensemble approaches

1. Problem Transformation

1.1 Binary Relevance:
This is the simplest technique, which basically treats each label as a separate single class classification problem. 
 
In binary relevance, this problem is broken into 4 different single class classification problems as shown in the figure below.

In [20]:
!pip install scikit-multilearn

     |████████████████████████████████| 92kB 4.8MB/s 


In [47]:
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(GaussianNB())

# train
classifier.fit(X_train, y_train)

# predict
predictions = classifier.predict(X_test)

we can’t simply use our normal metrics to calculate the accuracy of our predictions. For that purpose, we will use accuracy score metric. This function calculates subset accuracy meaning the predicted set of labels should exactly match with the true set of labels.

In [48]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predictions)

0.10468920392584515

**1.2 Classifier Chains**
the first classifier is trained just on the input data and then each next classifier is trained on the input space and all the previous classifiers in the chain.  

In [49]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain
from sklearn.naive_bayes import GaussianNB

# initialize classifier chains multi-label classifier
# with a gaussian naive bayes base classifier
classifier = ClassifierChain(GaussianNB())

# train
classifier.fit(X_train, y_train)

# predict
predictions = classifier.predict(X_test)

accuracy_score(y_test,predictions)

0.11777535441657579

**1.3 Label Powerset**
we transform the problem into a multi-class problem with one multi-class classifier is trained on all unique label combinations found in the training data.
All the multilabel columns are combined to form a multiclass column

In [50]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
from sklearn.naive_bayes import GaussianNB

# initialize Label Powerset multi-label classifier
# with a gaussian naive bayes base classifier
classifier = LabelPowerset(GaussianNB())

# train
classifier.fit(X_train, y_train)

# predict
predictions = classifier.predict(X_test)

accuracy_score(y_test,predictions)

0.18865866957470012

The only disadvantage of this is that as the training data increases, number of classes become more. Thus, increasing the model complexity, and would result in a lower accuracy.

**2. Adapted Algorithm**

adapting the algorithm to directly perform multi-label classification, rather than transforming the problem into different subsets of problems.

For example, multi-label version of kNN is represented by MLkNN. 

In [65]:
from skmultilearn.adapt import MLkNN

classifier = MLkNN(k=20)

# train
classifier.fit(X_train.to_numpy(), y_train.to_numpy())

# predict
predictions = classifier.predict(X_test)

accuracy_score(y_test,predictions)

0.1821155943293348

In [55]:
# Another example using adapted algorithm

from skmultilearn.adapt import MLkNN
from scipy import sparse
from skmultilearn.dataset import load_dataset
import sklearn.metrics as metrics

X_train, y_train, feature_names, label_names = load_dataset('emotions', 'train')
X_test, y_test, _, _ =load_dataset('emotions', 'test')

clf = MLkNN(k=5)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(type(y_pred))  # <class 'scipy.sparse.lil.lil_matrix'>

print(type(y_pred.toarray()))  # <class 'numpy.ndarray'>

y_pred_csr = sparse.csr_matrix(y_pred)

print(type(y_pred_csr))  # <class 'scipy.sparse.csr.csr_matrix'>

accuracy = metrics.accuracy_score(y_test, y_pred)
print(accuracy)  # 0.148

emotions:train - does not exists downloading
Downloaded emotions-train
emotions:test - does not exists downloading
Downloaded emotions-test
<class 'scipy.sparse.lil.lil_matrix'>
<class 'numpy.ndarray'>
<class 'scipy.sparse.csr.csr_matrix'>
0.1485148514851485


In [ ]:
# More at http://scikit.ml/multilabelembeddings.html